This is an example of how to train a basic image classification model with Keras and Tensorflow, using TileDB as storage for images.
First of all let's import everything we will need.

In [10]:
import tiledb
import os
import tensorflow as tf
import glob
import cv2
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Activation, MaxPooling2D, Dropout

We have to download the flower image dataset.

In [13]:
print("[STATUS] downloading image data...")
data_dir = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)
os.system("mv ~/.keras/datasets/flower_photos ./data")
print("[STATUS] downloading image data finished...")

[STATUS] downloading image data...
[STATUS] downloading image data finished...


Let's define image size in order to later rescale our images, batch size for training our model, data and TileDB array paths.

In [ ]:
# Image size for rescaling.
IMAGE_SIZE = (224, 224)

# Batch size for training an image classification model.
BATCH_SIZE = 32

# Where our data live.
DATA_PATH = "data/flower_photos"

# Where our tileDB arrays live.
TILEDB_PATH = "data/flower_photos_tiledb"

if not os.path.exists(TILEDB_PATH):
    os.mkdir(TILEDB_PATH)

# The actual TileDB Arrays.
tiledb_images_train = os.path.join(TILEDB_PATH, "images_train")
tiledb_labels_train = os.path.join(TILEDB_PATH, "labels_train")

tiledb_images_validate = os.path.join(TILEDB_PATH, "images_validate")
tiledb_labels_validate = os.path.join(TILEDB_PATH, "labels_validate")

tiledb_images_test = os.path.join(TILEDB_PATH, "images_test")
tiledb_labels_test = os.path.join(TILEDB_PATH, "labels_test")